**1. Tune the parameters of the ML-in-the-loop active learning workflow in order to find molecules with the largest ionization energy in the shortest possible time. Use the PBS submit script 4_submit_multinode.sh to run the full workflow on 1 or multiple nodes of Polaris. Note that all that should be needed for this exercise is to change the values of the initial_training_count, max_training_count and batch_size variables at the top of the 3_ml_in_the_loop.py script. Submit the plot that is produced by the script as well as your code to showcase your results and how you obtained them.**

![HW3](ml-in-the-loop/parsl_ml_in_the_loop.png)
* initial_training_count = 16  # Number of trianing samples to collect for first model training
* max_training_count = 64  # Maximum number of training samples to collect for training
* batch_size = 4  # Number of molecules to simulate in each iteration of active learning loop

**2. Experiment with the Parsl and DragonHPC implementations of the producer-consumer workflow by scaling up the problem size, both in terms of size of the data being produced/transferred and in terms of the number of nodes (although 2-4 nodes will be sufficient). Collect data from these experiments to fill in the table under the Data Transfer Performance (Homework) section at the bottom of the example README and write a short paragraph about your observations. See the notes under the Data Transfer Performance (Homework) section for more detailed information and some hints.**

| Nodes | # Sims | Grid Size | Data Size (GB) | **Simulation Run / IO (sec)** – Parsl FS | Dragon DDict | Parsl Futures | **Training Run / IO (sec)** – Parsl FS | Dragon DDict | Parsl Futures |
|------:|-------:|----------:|---------------:|-----------------------------------------:|-------------:|--------------:|----------------------------------------:|-------------:|---------------:|
| 1     | 64     | 512       | 0.62           | 11.22 / 0.094                            | **7.01 / 0.233** | 14.38 / NA    | **14.90 / 0.422**                       | 17.92 / 1.194 | 26.59 / NA     |
| 2     | 64     | 256       | 0.31           | 11.06 / 0.025                            | **7.08 / 0.100** | 11.45 / NA    | **14.67 / 0.984**                       | 27.74 / 1.182 | 23.86 / NA     |
| 2     | 64     | 512       | 1.25           | 11.27 / 0.112                            | **6.81 / 0.147** | 14.14 / NA    | **22.74 / 2.115**                       | 30.25 / 1.944 | 74.20 / NA     |
| 2     | 128    | 512       | 2.50           | 16.58 / 0.109                            | **12.42 / 0.150** | 19.83 / NA    | **23.71 / 2.122**                       | 30.98 / 2.117 | 161.11 / NA    |
| 2     | 64     | 1024      | 5.00           | 12.26 / 0.374                            | **7.84 / 0.659** | 23.10 / NA    | 56.33 / 6.067                            | **51.30 / 4.383** | 287.25 / NA    |

* At small data sizes (<1 GB), Parsl + file system gives the best training runtime and lowest training I/O, while Dragon DDict wins the simulation runtime.
* At medium data sizes (1.25–2.5 GB), the two are close: Dragon tends to have lower training I/O (and consistently the fastest simulation runtime), while Parsl + FS can still edge out total training runtime in some cases (e.g., 2.5 GB).
* At the largest case (5 GB), Dragon DDict clearly outperforms Parsl + FS on both training runtime and I/O, indicating that in-memory data exchange scales better than file-based exchange as volumes rise.
* Parsl + futures degrades sharply with scale, so it’s only reasonable for very small problems or single-node scenarios.